In [1]:
import torch 
from torch.utils.data import  DataLoader,datasets,Subset
from torchvision import transforms, datasets
from torchvision.models import resnet18
from tqdm import tqdm, trange

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [3]:
triplets_id = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()
test_triplets_id = pd.read_csv('task4_be9ai3nsdj/test_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()

In [4]:
train_id, validate_id = train_test_split(triplets_id, test_size=0.20, random_state=0, shuffle=True)

In [5]:
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(255),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])
dataset = datasets.ImageFolder('task4_be9ai3nsdj/', transform = transform)

In [6]:
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=False)

In [7]:
import torch.nn as nn
import torch.nn.functional as F

in_dim = 512 #features.shape[1]
out_dim = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.vision_model = resnet18()#.to(device)
        #self.hidden1 = nn.Linear(1000, 64) 
        #self.hidden2 = nn.Linear(512, 64)
        #self.hidden3 = nn.Linear(256, 128)
        self.triplet_loss = nn.TripletMarginLoss(margin=0.1, p=1.0, reduction='none')
        self.cosine_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)
        
    def forward(self, x):
        x = self.vision_model.forward(x)
        x = x.unsqueeze(0)
        #x = self.hidden1(x)
        
        #x = self.hidden2(x)
        #x = self.hidden3(x)
        x = self.triplet_loss(x[:,0,:], x[:,1,:], x[:,2,:])
            #- 2.0*self.cosine_similarity(x[:,0,:] - x[:,2,:], x[:,1,:] - x[:,2,:]) \
            #+ 0.3*self.cosine_similarity(x[:,0,:] - x[:,1,:], x[:,2,:] - x[:,1,:])
            #+ 1e-5*torch.tensordot(x[:,0,:] - x[:,1,:], x[:,2,:] - x[:,1,:], dims=2) \
            
        
        return x
    
    def predict(self, x):
        #x = self.hidden1(x)
        #x = self.hidden2(x)
        x = self.triplet_loss(x[:,0,:], x[:,1,:], x[:,2,:])
        
        return x
        

net = Net()#.to(device)

In [8]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [9]:
num_epochs = 1

In [10]:
i = 0
feature_list = []

pbar = trange(num_epochs)

for i in pbar:
    criterion = torch.Tensor([0])#.to(device)
    optimizer.zero_grad()

    for idx in triplets_id[:10]:
        image_triplet = torch.stack((dataset[idx[0]][0],
                                     dataset[idx[1]][0],
                                     dataset[idx[2]][0]))

        #image_triplet = image_triplet#.to(device)
        
        running_loss = 0.0

        # zero the parameter gradients

        # forward + backward + optimize
        outputs = net.forward(image_triplet)
        
        criterion += outputs
        
        del image_triplet
    
    criterion.backward()
    optimizer.step()

    pbar.set_description(f"mean {criterion}")

mean tensor([2311.7190], grad_fn=<AddBackward0>): 100%|██████████| 1/1 [00:28<00:00, 28.53s/it]
